In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import statsmodels.api as sm

In [6]:
fred_path = "/home/theo/code/Theo038/Forecasting-Gold-Price/raw_data/FRED_FEDFUNDS, 1M.csv"
hourly_path = "/home/theo/code/Theo038/Forecasting-Gold-Price/raw_data/Extract_TimeFrame_60_clean.csv"

In [5]:
# --- Load hourly file (semicolon; day-first dates) ---
df_h = pd.read_csv(hourly_path, sep=';')
df_h['time'] = pd.to_datetime(df_h['time'], dayfirst=True, errors='coerce', utc=True)
df_h = df_h.dropna(subset=['time']).sort_values('time').set_index('time')
df_h.head()


,open,high,low,close,Basis,Upper,Lower,Up Trend,Down Trend,KAMA,RSI,Bollinger Bands %b,Bollinger BandWidth,Highest Expansion,Lowest Contraction
time,,,,,,,,,,,,,,,
2022-01-11 00:00:00+00:00,1801.23,1802.85,1800.03,1801.18,1796.565,1802.85,1790.28,1790.975815,NaN,1795.256156,65.669045,0.831694,0.803134,2.519134,0.332677
2022-01-11 01:00:00+00:00,1801.16,1804.61,1801.15,1802.70,1797.445,1804.61,1790.28,1792.424234,NaN,1795.536505,69.806836,0.888896,0.826182,2.519134,0.332677
2022-01-11 02:00:00+00:00,1802.68,1805.91,1802.00,1804.85,1798.095,1805.91,1790.28,1793.371810,NaN,1795.896520,74.665739,0.963492,0.876434,2.519134,0.332677
2022-01-11 03:00:00+00:00,1804.83,1806.31,1804.21,1805.18,1798.295,1806.31,1790.28,1795.105129,NaN,1796.250762,75.350694,0.925586,0.917716,2.519134,0.332677
2022-01-11 04:00:00+00:00,1805.18,1807.73,1804.99,1805.91,1799.005,1807.73,1790.28,1796.398616,NaN,1796.635508,76.904655,0.915073,0.952947,2.519134,0.332677


In [21]:
# --- Load FRED monthly series ---
df_m = pd.read_csv(fred_path)
df_m['time'] = pd.to_datetime(df_m['time'], utc=True, errors='coerce')
df_m = df_m.dropna(subset=['time']).sort_values('time')
df_m = df_m.rename(columns={'close': 'FEDFUNDS'})
df_m.head()

,time,FEDFUNDS
0,1954-07-01 00:00:00+00:00,0.80
1,1954-08-01 00:00:00+00:00,1.22
2,1954-09-01 00:00:00+00:00,1.07
3,1954-10-01 00:00:00+00:00,0.85
4,1954-11-01 00:00:00+00:00,0.83


In [22]:
df_m.dtypes

time        datetime64[ns, UTC]
FEDFUNDS                float64
dtype: object

In [ ]:
# --- Propagate monthly values to all hourly rows within the month ---
# Strategy: as-of merge from month-start rows backward, so every hour in a month gets that month's value.
# (Equivalent to repeating constant within month; avoids per-day noise)
df_h_reset = df_h.copy().reset_index()
df_m_reset = df_m.copy().rename(columns={'month_start': 'time'})
df_merged = pd.merge_asof(
    df_h_reset.sort_values('time'),
    df_m_reset.sort_values('time'),
    on='time',
    direction='backward'
).set_index('time')
df_merged.head(900)


,open,high,low,close,Basis,Upper,Lower,Up Trend,Down Trend,KAMA,RSI,Bollinger Bands %b,Bollinger BandWidth,Highest Expansion,Lowest Contraction,FEDFUNDS
time,,,,,,,,,,,,,,,,
2022-01-11 00:00:00+00:00,1801.23,1802.85,1800.03,1801.18,1796.565,1802.85,1790.28,1790.975815,NaN,1795.256156,65.669045,0.831694,0.803134,2.519134,0.332677,0.08
2022-01-11 01:00:00+00:00,1801.16,1804.61,1801.15,1802.70,1797.445,1804.61,1790.28,1792.424234,NaN,1795.536505,69.806836,0.888896,0.826182,2.519134,0.332677,0.08
2022-01-11 02:00:00+00:00,1802.68,1805.91,1802.00,1804.85,1798.095,1805.91,1790.28,1793.371810,NaN,1795.896520,74.665739,0.963492,0.876434,2.519134,0.332677,0.08
2022-01-11 03:00:00+00:00,1804.83,1806.31,1804.21,1805.18,1798.295,1806.31,1790.28,1795.105129,NaN,1796.250762,75.350694,0.925586,0.917716,2.519134,0.332677,0.08
2022-01-11 04:00:00+00:00,1805.18,1807.73,1804.99,1805.91,1799.005,1807.73,1790.28,1796.398616,NaN,1796.635508,76.904655,0.915073,0.952947,2.519134,0.332677,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-07 02:00:00+00:00,1988.25,2001.03,1987.84,1997.78,1967.475,2001.03,1933.92,1969.343705,NaN,1968.752263,93.137826,1.058138,3.657488,3.657488,0.527837,0.20
2022-03-07 03:00:00+00:00,1997.83,1998.18,1986.75,1986.92,1967.475,2001.03,1933.92,1969.343705,NaN,1970.752531,71.917095,0.864458,3.713897,3.713897,0.527837,0.20
2022-03-07 04:00:00+00:00,1987.02,1990.51,1984.67,1990.02,1967.475,2001.03,1933.92,1969.343705,NaN,1972.942613,73.831762,0.866296,3.744396,3.744396,0.527837,0.20


In [30]:
def chronological_split(X, y, train_ratio=0.7):
    """Chronological split (no shuffle). Returns X_train, X_val, y_train, y_val."""
    X = X.sort_index()
    y = y.loc[X.index]
    split_idx = int(np.floor(len(X) * train_ratio))
    return X.iloc[:split_idx], X.iloc[split_idx:], y.iloc[:split_idx], y.iloc[split_idx:]

def evaluate_metrics(y_true, y_pred, prefix=""):
    """Compute MAE, RMSE, R2 and print them."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"{prefix} MAE={mae:.6f} | RMSE={rmse:.6f} | R2={r2:.6f}")
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

def learning_curves_time_series(estimator, X, y, train_sizes, n_splits=5, scoring='rmse'):
    """
    Time-series learning curves: for increasing train sizes, use TimeSeriesSplit folds.
    Returns arrays of avg train and validation scores.
    """
    train_scores = []
    val_scores = []

    for ts in train_sizes:
        X_sub = X.iloc[:ts]
        y_sub = y.iloc[:ts]

        tscv = TimeSeriesSplit(n_splits=n_splits)
        fold_train_scores, fold_val_scores = [], []

        for tr_idx, va_idx in tscv.split(X_sub):
            X_tr, X_va = X_sub.iloc[tr_idx], X_sub.iloc[va_idx]
            y_tr, y_va = y_sub.iloc[tr_idx], y_sub.iloc[va_idx]

            est = clone(estimator)
            est.fit(X_tr, y_tr)

            y_tr_pred = est.predict(X_tr)
            y_va_pred = est.predict(X_va)

            if scoring == 'rmse':
                tr = root_mean_squared_error(y_tr, y_tr_pred)
                va = root_mean_squared_error(y_va, y_va_pred)
            elif scoring == 'mae':
                tr = mean_absolute_error(y_tr, y_tr_pred)
                va = mean_absolute_error(y_va, y_va_pred)
            elif scoring == 'r2':
                tr = r2_score(y_tr, y_tr_pred)
                va = r2_score(y_va, y_va_pred)
            else:
                raise ValueError("Unsupported scoring")

            fold_train_scores.append(tr)
            fold_val_scores.append(va)

        train_scores.append(np.mean(fold_train_scores))
        val_scores.append(np.mean(fold_val_scores))

    return np.array(train_scores), np.array(val_scores)


In [31]:
class TrendEngineeringTransformer(BaseEstimator, TransformerMixin):
    """
    Derive 'Trend' (categorical: 'Up'/'Down') from 'Up Trend' / 'Down Trend',
    derive 'Trend_value' as row-wise sum of the two (skipna), and optionally drop originals.
    """
    def __init__(self, up_col='Up Trend', down_col='Down Trend', out_cat_col='Trend',
                 out_num_col='Trend_value', drop_original=True):
        self.up_col = up_col
        self.down_col = down_col
        self.out_cat_col = out_cat_col
        self.out_num_col = out_num_col
        self.drop_original = drop_original

    def fit(self, X, y=None):
        # Nothing to learn
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("TrendEngineeringTransformer expects a pandas DataFrame.")

        X = X.copy()
        # If columns are missing, create them filled with NaN so downstream logic remains consistent
        if self.up_col not in X.columns:
            X[self.up_col] = np.nan
        if self.down_col not in X.columns:
            X[self.down_col] = np.nan

        # 'Trend' = 'Up' if Up Trend not NaN else 'Down'
        X[self.out_cat_col] = np.where(X[self.up_col].notna(), 'Up', 'Down')
        # 'Trend_value' = Up Trend + Down Trend (skipna)
        X[self.out_num_col] = X[[self.up_col, self.down_col]].sum(axis=1, skipna=True)

        # Drop originals if requested
        if self.drop_original:
            X = X.drop(columns=[self.up_col, self.down_col], errors='ignore')

        return X


class CustomFeatureSelector(BaseEstimator, TransformerMixin):
    """
    Drop highly correlated numerical features (> threshold) based on upper triangle of correlation matrix.
    """
    def __init__(self, num_corr_threshold=0.95, method='pearson'):
        self.num_corr_threshold = num_corr_threshold
        self.method = method

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("CustomFeatureSelector expects a pandas DataFrame.")
        self.num_cols_ = list(X.select_dtypes(include=[np.number]).columns)
        if len(self.num_cols_) == 0:
            self.num_col_to_drop_ = []
            return self
        corr_num = X[self.num_cols_].corr(method=self.method)
        upper = corr_num.where(np.triu(np.ones(corr_num.shape), k=1).astype(bool)).abs()
        self.num_col_to_drop_ = [c for c in upper.columns if any(upper[c] > self.num_corr_threshold)]
        return self

    def transform(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("CustomFeatureSelector expects a pandas DataFrame.")
        return X.drop(columns=getattr(self, 'num_col_to_drop_', []), errors='ignore')

In [32]:
def build_preprocessing_pipeline(
    monthly_cols=('FEDFUNDS',),    # add more monthly series here if you merge others
    num_corr_threshold=0.95,
    trend_up_col='Up Trend',
    trend_down_col='Down Trend',
    trend_out_cat='Trend',
    trend_out_num='Trend_value'
):
    # Dynamic numeric selector (after feature dropper); categorical encoder explicitly on 'Trend'
    num_selector = make_column_selector(dtype_include=np.number)
    cat_cols = [trend_out_cat]

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_selector),
            ('cat', OrdinalEncoder(categories=[['Down', 'Up']],
                                   handle_unknown='use_encoded_value', unknown_value=np.nan),
             cat_cols),
        ],
        remainder='drop'
    )

    pipe_new = Pipeline(steps=[
        ('trend_engineer', TrendEngineeringTransformer(up_col=trend_up_col, down_col=trend_down_col,
                                                      out_cat_col=trend_out_cat, out_num_col=trend_out_num,
                                                      drop_original=True)),
        ('corr_feature_dropper', CustomFeatureSelector(num_corr_threshold=num_corr_threshold, method='pearson')),
        ('preprocess', preprocessor),
    ])
    return pipe_new


In [ ]:
all_cols = list(df_merged.columns)
target_col = "close"
feature_cols = [c for c in all_cols if c != target_col]

X = df_merged[feature_cols]
y = df_merged[target_col]

In [ ]:
df = df.sort_index().copy()
target_col = f"{target_source_col}_t+{horizon_hours}"
df[target_col] = df[target_source_col].shift(-horizon_hours)
df = df.dropna(subset=[target_col])

feature_cols = hourly_cols + monthly_cols
X = df[feature_cols].copy()
y = df[target_col].copy()

In [34]:
X_train_raw, X_val_raw, y_train, y_val = chronological_split(X, y, train_ratio=0.7)

In [ ]:
pipe_new = build_preprocessing_pipeline(
    hourly_cols=hourly_cols,
    monthly_cols=monthly_cols,
    lag_months=1,
    num_corr_threshold=0.95
)

# Model zoo
models = {
    'model_gbm': GradientBoostingRegressor(random_state=42),
    'model_rf': RandomForestRegressor(random_state=42),
    'model_dt': DecisionTreeRegressor(random_state=42),
    'model_svm': SVR(),
    'model_ada': AdaBoostRegressor(random_state=42),
	'model_xgb': XGBRegressor(random_state=42, n_jobs=-1)

}

params = {
    'model_gbm': {'model__learning_rate':[0.1,0.2,0.3,0.4], 'model__n_estimators':[50,100,500,1000,2000], 'model__max_depth':[3,5,7]},
    'model_rf':  {'model__n_estimators':[50,100,500,1000,2000], 'model__max_depth':[2,5,7,10,20]},
    'model_dt':  {'model__splitter':['best','random'], 'model__max_depth':[2,5,7,10,20,50]},
    'model_svm': {'model__C':[1,2,5,10,50,100,500], 'model__kernel':['rbf','poly','sigmoid','linear'], 'model__degree':[2,3,4], 'model__gamma':['scale','auto']},
    'model_ada': {'model__n_estimators':[10,20,30,50,100,500,1000], 'model__learning_rate':[0.5,1,2,5,10]},
	'model_xgb': {'model__n_estimators':[50,100,500,1000], 'model__learning_rate':[0.01,0.05,0.1,0.2], 'model__max_depth':[3,5,7,10]}
}

# Time-series CV & scoring (RMSE)
tscv = TimeSeriesSplit(n_splits=5)
scoring = 'neg_root_mean_squared_error'  # robust for (possibly negative) returns

best_models = {}
for name, model in models.items():
    final_pipe = Pipeline([
        ('preprocessing', pipe_new),
        ('model', model)
    ])
    grid = params.get(name, {})
    search = GridSearchCV(final_pipe, grid, scoring=scoring, cv=tscv, n_jobs=-1, verbose=1)
    search.fit(X, y)  # X=df[hourly+monthly], y=target

    best_models[name] = search.best_estimator_
    print(f"{name}: Best params -> {search.best_params_}, Score -> {search.best_score_:.6f}")